# Sheet
Vergelijk de verschillen in de NBS per issue en per waarde van p

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import ipywidgets as widget
from IPython.core.display import display, HTML
%matplotlib inline

pd.set_option('display.max_rows', 750)
pd.set_option('display.max_columns', 500)

In [2]:
conn = sqlite3.connect('/home/jelmert/PycharmProjects/decide-exchange-model/data/output/decide-data_1.db')

In [3]:
df = pd.read_sql("""
SELECT
        a.name AS actor,
        i.name as issue,
        AVG(ai.position) as postion,        
        i2.pointer                                AS iteration,
        m.p,
        m.id  
      FROM actorissue ai
        LEFT JOIN issue i ON ai.issue_id = i.id
        LEFT JOIN actor a ON ai.actor_id = a.id
        LEFT JOIN iteration i2 ON ai.iteration_id = i2.id
        LEFT JOIN repetition r ON i2.repetition_id = r.id
        LEFT JOIN modelrun m ON r.model_run_id = m.id
        LEFT JOIN dataset d ON a.data_set_id = d.id
      WHERE  ai.type = 0 AND d.id = ?
     GROUP BY m.id, i2.pointer, a.id, i.id;
""", 
                 conn, 
                 params=(1,),
                 index_col=['actor', 'issue','p'],                  
                 columns=['postion']
                )

table = pd.pivot_table(df, index=['actor', 'issue','iteration'], columns=['p'],values=['postion'])


DatabaseError: Execution failed on sql '
SELECT
        a.name AS actor,
        i.name as issue,
        AVG(ai.position) as postion,        
        i2.pointer                                AS iteration,
        m.p,
        m.id  
      FROM actorissue ai
        LEFT JOIN issue i ON ai.issue_id = i.id
        LEFT JOIN actor a ON ai.actor_id = a.id
        LEFT JOIN iteration i2 ON ai.iteration_id = i2.id
        LEFT JOIN repetition r ON i2.repetition_id = r.id
        LEFT JOIN modelrun m ON r.model_run_id = m.id
        LEFT JOIN dataset d ON a.data_set_id = d.id
      WHERE  ai.type_int = 0 AND d.id = ?
     GROUP BY m.id, i2.pointer, a.id, i.id;
': no such column: ai.type_int

# Posities 
Verloop van de positie over de verschillende onderhandelingsrondes van de actor per issue 

In [ ]:
table

In [ ]:
cursor = conn.execute('SELECT `name`, `id`, lower, upper FROM issue ORDER BY `name`')
issues = cursor.fetchall()

cursor = conn.execute('SELECT `name`, `id` FROM actor ORDER BY `name`')
actors = cursor.fetchall()

for actor, actor_id in actors:
    
    fig, axes = plt.subplots(3, 2, figsize=(15,15))  # a figure with a 2x2 grid of Axes    
    i = 0
    fig.delaxes(axes[2,1])
    display(HTML('<hr><h1><center>{}</center></h1>'.format(actor)))
    for issue in issues:
        
        row = int(i / 2)
        col = i % 2
        
        subset = table.query('actor == ["{}"] and issue == ["{}"]'.format(actor, issue[0]))
        subchart = axes[row, col]
        subchart.plot(subset.values)
        subchart.set_ylim(0, 110)        
        subchart.set_title(issue[0])
        subchart.set_ylabel('Issue score')
        subchart.set_xlabel('Negotiation round (iteration)')
        
        i+=1
        
    
    plt.tight_layout()
    plt.show()
    